# Unit 8 lab : Graph algorithms

This week we are looking at graphs / networks and a number of algorithms that can be applied to them.

In the example below, mygraph is a dictionary storing an adjacency list representation of a graph.  It might help to draw it on paper.


In [ ]:
mygraph={'A':['B','C','G'],'B':['A','D','H'],'C':['A'],'D':['B','E','G'],'E':['D','H'],'F':['H'],'G':['D','H'],'H':['B','E','F']}

## Breadth-first search

BFS is an implementation of the breadth-first search algorithm introduced in the lecture

In [ ]:
import numpy as np

def BFS(graphdict,source):
    
    #initialisation
    colour={}
    distance={}
    predecessor={}
    for v in graphdict.keys():
        colour[v]='w'
        distance[v]=np.inf
        predecessor[v]=None
        
    colour[source]='g'
    distance[source]=0
    queue=[source]
    
    #iteration
    while queue != []:
        u=queue[0]
        for vertex in graphdict[u]:
            if colour[vertex]=='w':
                colour[vertex]='g'
                distance[vertex]=distance[u]+1
                predecessor[vertex]=u
                queue.append(vertex)
        colour[u]='b'
        
        if len(queue)>1:
            queue=queue[1:]
        else:
            queue=[]
            
    return (distance,predecessor)
        

### Exercise 1a
Run BFS() with different sources.  
1. What is the shortest distance from A to G?
2. What is the shortest distance from A to F?
3. What is the shortest distance from B to E?


### Exercise 1b
Write a function to output any pairs of nodes where the length of the shortest path is greater than n.  Use it to find a list of all pairs where the length of the shortest path is greater than 3.

### Exercise 1c
Write a function which takes the second output of the BFS function (i.e., the predecessor dictionary) and writes out the path from the source to a given vertex.  What is the shortest path from A to E?

## Dijkstra's algorithm

Dijkstra's algorithm is used to find the shortest paths from a source vertex in a weighted graph.

It makes use of a data structure known as a priority queue which maintains a list of items in order of increasing weight or priority.

A priority queue can be initialised with a list of items (where each item is a (name,priority) pair).  This list will be sorted by priority/weight.  It then supports 
1. the extractmin() operation which removes and returns the item with the lowest weight.
2. the insert(item) operation which adds an item to the queue (replacing an existing item with the same name)
3. the display() operation which displays the current state of the queue



In [ ]:
from operator import itemgetter

class queue:
    #this is a data structure to implement a simple priority queue
    
    def __init__(self,alist):
        self.items=alist
        self.items=sorted(self.items,key=itemgetter(1))  #ensure that the initial list is sorted by the weight
        
    def extractmin(self):
        #remove and return the item with the lowest weight
        #as self.items is maintained in order of weight, this is the first item
        
        if len(self.items)>0:
            res=self.items[0]
            if len(self.items)>1:
                self.items=self.items[1:]
            else:
                self.items=[]
        
        else:
            res=None
        
        return res
    
    def remove(self,delitem):
        # remove an item which matches the given item's name (ignoring priority)
        delindex=-1
        
        for i,item in enumerate(self.items):
            if item[0]==delitem[0]:
                delindex=i
        
        if delindex>-1:
            if delindex<len(self.items)-1:
                self.items=self.items[0:delindex]+self.items[delindex+1:]
            else:
                self.items=self.items[0:delindex]
       
    def insert(self,newitem):
        
        #remove the item with the same name 
        self.remove(newitem) 
        
        #insert the item in the correct place in the priority queue        
        index=-1
        for i,item in enumerate(self.items):
            if item[1] > newitem[1]:
                index=i
                break
        
        if index>-1:       
            self.items=self.items[0:index]+[newitem]+self.items[index:]
        else:
            self.items.append(newitem)
    
    def display(self):
        print(self.items)
        
        
### Test this by initialising a queue with some items and then using the different methods  



### Exercise 2a (Optional)
What is the time complexity of queue.extractmin()?
What is the time complexity of queue.insert()?  Give your answers in big O notation.
(Note this is a very naive implementation of a priority queue.  It is possible to do much better if self.items is stored in a data structure called a heap)

Dijkstra's algorithm is then much like BFS.  However, rather than adding vertices to the queue in the order they are discovered, it adds them with a priority according to their current distance from the source.

The minimum item in the queue, u, cannot have any further reductions in its distance (since negative weights are not allowed) so at each iteration, the minimum item from the queue, u,  is extracted and the outgoing edges from this vertex are **relaxed** i.e., it is checked whether the distance from the source to each adjacent vertex of u is reduced by going via u - if so the current distance of the distance to the adjacent vertex is updated and the priority queue is also updated.

In [ ]:
weightedgraph={'A':[('B',5),('C',2),('G',1)],'B':[('A',5),('D',3),('H',1)],'C':[('A',2)],'D':[('B',3),('E',5),('G',1)],'E':[('D',5),('H',2)],'F':[('H',1)],'G':[('D',1),('H',2)],'H':[('B',1),('E',2),('F',1)]}

def dijkstra(graphdict,source):
    
    #initialisation 
    #- much like BFS, however we don't need to colour the vertices any more
    #- because a vertex can be discovered more than once (it may be a shorter distance on subsequent discovery)
    
    distance={}
    predecessor={}
    for v in graphdict.keys():
        distance[v]=np.inf
        predecessor[v]=None
    distance[source]=0
    completed=[]
    
    #initialise a priority queue with the source vertex and its distance from the source (0)
    thequeue=queue([(source,0)])
    
    #this iteration will stop when all vertices added to completed
    while len(completed)<len(graphdict.keys()):
        
        #the item in the queue with the highest priority (lowest distance)
        #cannot have its distance reduced any further (assuming no negative weights)
        #so extract this and add it to the completed list
        (u,_)=thequeue.extractmin() 
        completed.append(u)
        
        #consider updating distances for all vertices adjacent to u
        #this is known as 'relaxing' the edges out of u
        for (v,weight) in graphdict[u]:
            if distance[v]> distance[u]+weight:
                #if distance to v is less via u then update the distance matrix
                #and add v to the priority queue with new distance as priority
                distance[v] = distance[u]+weight
                predecessor[v] = u
                thequeue.insert((v,distance[v]))
        
        #thequeue.display()
                
        
    return(distance,predecessor,completed)
    
    
dijkstra(weightedgraph,'A')        

### Exercise 2b

Use your code from earlier (modify if necessary) to output the shortest path from A to E.

### Exercise 2c (optional)
Given the naive implementation of the priority queue, what is the time complexity of Dijkstra's algorithm?

An alternative implementation of a priority queue uses a binary heap.  In this implementation, the time complexity of extractmin() is O(log n) and the complexity of insert() is O(log n).  How does this change the overall running time of Dijkstra's algoritm?